In [1]:
# Data to predict
predicton_data = {
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0,
    "MonthlyIncome" : 100000
}

In [2]:
# On importe la librairie bigml et pandas
from bigml.api import BigML
import pandas as pd
# Importe les fonctions depuis my_functions.ipynb
import ipynb.fs.full.my_functions as myfn

In [3]:
# On se connecte à bigml et on récupère un ensemble
api = BigML(project='project/5d94a32e42129f2e16000232')
ensemble = api.get_ensemble('ensemble/5dc2f3f07811dd7f2d012312')

In [4]:
# On récupère le dataset utilisé pour entrainer le modèle
dataset = api.get_dataset(ensemble['object']['dataset'])

In [5]:
# On récupère l'ensemble des colonnes du dataset
dataset_columns = []
for row in dataset['object']['fields']:
    dataset_columns.append(dataset['object']['fields'][row]['name'])

In [6]:
# On crée un dataframe pandas et on injecte les colonnes
df = pd.DataFrame(columns=dataset_columns)
# puis les données
df = df.append(predicton_data,ignore_index=True)

In [7]:
# Applique les prétraitements sur le dataframe
df = myfn.pretraitements(df)

In [8]:
# Transforme le dataframe en dictionnaire
data = df.iloc[0].to_dict()

In [9]:
myfn.bprint(data)

{   'DebtRatio': 0.0,
    'Id': 'NA',
    'IsOld': 0.0,
    'MonthlyIncome': 100000.0,
    'NumberOfDependents': 1.0,
    'NumberOfOpenCreditLinesAndLoans': 6.0,
    'NumberOfTimeGlobal': 1.0,
    'NumberRealEstateLoansOrLines': 1.0,
    'RevolvingUtilizationOfUnsecuredLines': 0.01703559,
    'SeriousDlqin2yrs': 'NA',
    'age': 42.0}


In [10]:
# Requête BigMl pour obtenir la prediction
prediction = api.create_prediction(ensemble, data, {"explain" : True, "name": "TEST 100000$ monthly income"})
api.ok(prediction)

True

In [11]:
if prediction['object']['probabilities'][0][1] > 0.5:
    print(f"Prediction is {prediction['object']['probabilities'][0][0]} with a probability of {prediction['object']['probabilities'][0][1]*100}%")
else:
    print(f"Prediction is {prediction['object']['probabilities'][1][0]} with a probability of {prediction['object']['probabilities'][1][1]*100}%")

Prediction is 0 with a probability of 97.129%


In [19]:
api.pprint(prediction['object']['explanation'])

[   {'field': '00000d', 'importance': 0.90565, 'operator': '<', 'value': 2},
    {'field': '00000d', 'importance': 0.06236, 'operator': '>', 'value': 0},
    {'field': '000009', 'importance': 0.02035, 'operator': '<', 'value': 2},
    {'field': '000002', 'importance': 0.0071, 'operator': '<', 'value': 45}]


In [17]:
for row in prediction['object']['fields']:
    print(prediction['object']['fields'][row]['name'])

RevolvingUtilizationOfUnsecuredLines
age
IsOld
DebtRatio
MonthlyIncome
NumberOfOpenCreditLinesAndLoans
NumberRealEstateLoansOrLines
NumberOfDependents
SeriousDlqin2yrs
NumberOfTimeGlobal
